### Grupo 4
### Miguel Egido Morales, Ana Robledano Abasolo, Alfredo Robledano Abasolo
# P3 AA Redes de Neuronas Convolucionales

In [ ]:
import tensorflow as tf

In [95]:
import tensorflow_datasets as tfds
(x1_train, y1_train), (x1_test, y1_test) = tf.keras.datasets.cifar100.load_data(label_mode='coarse')
(x2_train, y2_train), (x2_test, y2_test) = tf.keras.datasets.cifar100.load_data(label_mode='fine')

In [127]:
(ds_train, ds_test) , ds_info = tfds.load('cifar100', split=['train', 'test'], with_info=True)
my_np = tfds.as_numpy(ds_train)
for i in ds_train:
  print(i)
  break
i['coarse_label']

{'coarse_label': <tf.Tensor: shape=(), dtype=int64, numpy=12>, 'id': <tf.Tensor: shape=(), dtype=string, numpy=b'train_16399'>, 'image': <tf.Tensor: shape=(32, 32, 3), dtype=uint8, numpy=
array([[[151, 154, 145],
        [144, 147, 135],
        [141, 143, 134],
        ...,
        [ 44,  38,  39],
        [112, 100,  96],
        [145, 127, 120]],

       [[153, 156, 150],
        [141, 144, 135],
        [139, 142, 136],
        ...,
        [ 68,  59,  59],
        [131, 113, 106],
        [121, 101,  94]],

       [[163, 165, 161],
        [150, 153, 145],
        [147, 149, 144],
        ...,
        [ 75,  66,  60],
        [101,  86,  77],
        [ 83,  77,  73]],

       ...,

       [[150, 150, 162],
        [ 51,  52,  67],
        [ 46,  42,  56],
        ...,
        [ 60,  59,  48],
        [ 71,  73,  56],
        [ 58,  62,  44]],

       [[146, 158, 168],
        [ 92, 106, 116],
        [ 73,  77,  85],
        ...,
        [ 54,  55,  40],
        [ 59,  66,  41],
 

<tf.Tensor: shape=(), dtype=int64, numpy=12>